In [1]:
import os

import torch
import numpy as np
import pandas as pd
import argparse

from dataloaders.dataloaders import make_dataset
from dataloaders.preprocessing import preprocessing_cat, preprocessing_num

/home/connectome/jubin/.conda/envs/3DCNN/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
command = '--transfer MAE --num_target age --dataset ABCD --data freesurfer --val_size 0.1 --test_size 0.1 --lr 1e-5 --optim Adam --resize 96 96 96 --scheduler on --train_batch_size 112 --val_batch_size 112 --exp_name datatest --model densenet3D121 --epoch 3 --epoch_FC 0 --unfrozen_layer all --gpus 0'
command = command.split()

In [3]:
parser = argparse.ArgumentParser()

#parser.add_argument("--GPU_NUM",default=1,type=int,required=True,help='')
parser.add_argument("--model",required=True,type=str,help='',choices=['simple3D','vgg3D11','vgg3D13','vgg3D16','vgg3D19','resnet3D50','resnet3D101','resnet3D152', 'densenet3D121', 'densenet3D169','densenet201','densenet264'])
parser.add_argument("--dataset",required=True, type=str, choices=['UKB','ABCD'],help='') # revising
parser.add_argument("--data", type=str, help='select data type') # revising
parser.add_argument("--val_size",default=0.1,type=float,required=False,help='')
parser.add_argument("--test_size",default=0.1,type=float,required=False,help='')
parser.add_argument("--resize",default=(96, 96, 96),type=int,nargs="*",required=False,help='')
parser.add_argument("--train_batch_size",default=16,type=int,required=False,help='')
parser.add_argument("--val_batch_size",default=16,type=int,required=False,help='')
parser.add_argument("--test_batch_size",default=1,type=int,required=False,help='')
parser.add_argument("--in_channels",default=1,type=int,required=False,help='')
parser.add_argument("--optim",type=str,required=True,help='', choices=['Adam','SGD','RAdam','AdamW'])
parser.add_argument("--scheduler",type=str,default='',help='') # revising
parser.add_argument("--lr", default=0.01,type=float,required=False,help='')
parser.add_argument("--lr_adjust", default=0.01, type=float, required=False,help='')   
parser.add_argument("--weight_decay",default=0.001,type=float,required=False,help='')
parser.add_argument("--epoch",type=int,required=True,help='')
parser.add_argument("--epoch_FC",type=int,required=False,default=0,help='')
parser.add_argument("--exp_name",type=str,required=True,help='')
parser.add_argument("--cat_target", type=str, nargs='*', required=False, help='')
parser.add_argument("--num_target", type=str,nargs='*', required=False, help='')
parser.add_argument("--confusion_matrix", type=str, nargs='*',required=False, help='')
parser.add_argument("--gpus", type=int,nargs='*', required=False, help='')
parser.add_argument("--sbatch", type=str, required=False, choices=['True', 'False'])
parser.add_argument("--transfer", type=str, required=False, default="", choices=['sex','age','simclr','MAE'])
parser.add_argument("--unfrozen_layer", type=str, required=False, default='0') 
parser.add_argument("--load", type=str, required=False, default="")
parser.add_argument("--init_unfrozen", type=str, required=False, default="",help='init unfrozen layers')

_StoreAction(option_strings=['--init_unfrozen'], dest='init_unfrozen', nargs=None, const=None, default='', type=<class 'str'>, choices=None, help='init unfrozen layers', metavar=None)

In [4]:
args = parser.parse_args(args=command)
print("*** Categorical target labels are {} and Numerical target labels are {} *** \n".format(
    args.cat_target, args.num_target)
     )

if not args.cat_target:
    args.cat_target = []
elif not args.num_target:
    args.num_target = []
elif not args.cat_target and args.num_target:
    raise ValueError('YOU SHOULD SELECT THE TARGET!')

*** Categorical target labels are None and Numerical target labels are ['age'] *** 



In [5]:
cwd = os.getcwd()

## ========= Setting ========= ##
# seed number
seed = 1234
np.random.seed(seed)
torch.manual_seed(seed)


if args.transfer in ['age','MAE']:
    assert 96 in args.resize
else:
    assert 80 in args.resize

save_dir = os.getcwd() + '/result' #  
partition, subject_data = make_dataset(args) #  

/scratch/connectome/jubin/ABCD-3DCNN/STEP_5_Transfer_learning/dataloaders/dataloaders.py:52: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  image_files = pd.concat([image_files, pd.Series(glob.glob('*.nii.gz'))])
100%|██████████████████████████████████| 11357/11357 [00:00<00:00, 80521.57it/s]

*** Making a dataset is completed *** 



In [7]:
subject_data.age.describe()

count    11536.000000
mean       118.953190
std          7.464702
min        108.000000
25%        112.000000
50%        119.000000
75%        125.000000
max        131.000000
Name: age, dtype: float64

In [8]:
((subject_data.age-118.953190)/7.464702).describe()

count    1.153600e+04
mean     1.858027e-09
std      1.000000e+00
min     -1.467331e+00
25%     -9.314759e-01
50%      6.270846e-03
75%      8.100538e-01
max      1.613837e+00
Name: age, dtype: float64

In [10]:
0.8*7.464 

5.9712000000000005